<img src = "https://i.imgur.com/UjutVJd.jpg" align = "center">





# Applying Recurrent Neural Networks/LSTM for Language Modelling

Hello and welcome to this part. In this notebook, we will go over the topic of what Language Modelling is and create a Recurrent Neural Network model based on the Long Short-Term Memory unit to train and be benchmarked by the Penn Treebank. By the end of this notebook, you should be able to understand how TensorFlow builds and executes a RNN model for Language Modelling.

<hr>

## The Objective
By now, you should have an understanding of how Recurrent Networks work -- a specialized model to process sequential data by keeping track of the "state" or context. In this notebook, we go over a TensorFlow code snippet for creating a model focused on **Language Modelling** -- a very relevant task that is the cornerstone of many different linguistic problems such as **Speech Recognition, Machine Translation and Image Captioning**. For this, we will be using the Penn Treebank, which is an often-used dataset for benchmarking Language Modelling models.

## What exactly is Language Modelling?
Language Modelling, to put it simply, **is the task of assigning probabilities to sequences of words**. This means that, given a context of one or a few words in the language the model was trained on, the model should have a knowledge of what are the most probable words or sequence of words for the sentence. Language Modelling is one of the tasks under Natural Language Processing, and one of the most important.

<img src=https://ibm.box.com/shared/static/1d1i5gub6wljby2vani2vzxp0xsph702.png width="768"/>
<center>*Example of a sentence being predicted*</center>

In this example, one can see the predictions for the next word of a sentence, given the context "This is an". As you can see, this boils down to a sequential data analysis task -- you are given a word or a sequence of words (the input data), and, given the context (the state), you need to find out what is the next word (the prediction). This kind of analysis is very important for language-related tasks such as **Speech Recognition, Machine Translation, Image Captioning, Text Correction** and many other very relevant problems. 

<img src=https://ibm.box.com/shared/static/az39idf9ipfdpc5ugifpgxnydelhyf3i.png width="1080"/>
<center>*The above example schematized as an RNN in execution*</center>

As the above image shows, Recurrent Network models fit this problem like a glove. Alongside LSTM and its capacity to maintain the model's state for over one thousand time steps, we have all the tools we need to undertake this problem. The goal for this notebook is to create a model that can reach **low levels of perplexity** on our desired dataset.

For Language Modelling problems, **perplexity** is the way to gauge efficiency. Perplexity is simply a measure of how well a probabilistic model is able to predict its sample. A higher-level way to explain this would be saying that **low perplexity means a higher degree of trust in the predictions the model makes**. Therefore, the lower perplexity is, the better.

## The Penn Treebank dataset
Historically, datasets big enough for Natural Language Processing are hard to come by. This is in part due to the necessity of the sentences to be broken down and tagged with a certain degree of correctness -- or else the models trained on it won't be able to be correct at all. This means that we need a **large amount of data, annotated by or at least corrected by humans**. This is, of course, not an easy task at all.

The Penn Treebank, or PTB for short, is a dataset maintained by the University of Pennsylvania. It is *huge* -- there are over **four million and eight hundred thousand** annotated words in it, all corrected by humans. It is composed of many different sources, from abstracts of Department of Energy papers to texts from the Library of America. Since it is verifiably correct and of such a huge size, the Penn Treebank has been used time and time again as a benchmark dataset for Language Modelling.

The dataset is divided in different kinds of annotations, such as Piece-of-Speech, Syntactic and Semantic skeletons. For this example, we will simply use a sample of clean, non-annotated words (with the exception of one tag -- `<unk>`, which is used for rare words such as uncommon proper nouns) for our model. This means that we just want to predict what the next words would be, not what they mean in context or their classes on a given sentence. 
<br/>
<div class="alert alert-block alert-info" style="margin-top: 20px">
<center>the percentage of lung cancer deaths among the workers at the west `<unk>` mass. paper factory appears to be the highest for any asbestos workers studied in western industrialized countries he said 
 the plant which is owned by `<unk>` & `<unk>` co. was under contract with `<unk>` to make the cigarette filters 
 the finding probably will support those who argue that the u.s. should regulate the class of asbestos including `<unk>` more `<unk>` than the common kind of asbestos `<unk>` found in most schools and other buildings dr. `<unk>` said</center>
</div>
 <center>*Example of text from the dataset we are going to use, `ptb.train`*</center>
 <br/>



<h2>Word Embeddings</h2><br/>

For better processing, in this example, we will make use of [**word embeddings**]( [https://www.tensorflow.org/tutorials/word2vec/), which are **a way of representing sentence structures or words as n-dimensional vectors (where n is a reasonably high number, such as 200 or 500) of real numbers**. Basically, we will assign each word a randomly-initialized vector, and input those into the network to be processed. After a number of iterations, these vectors are expected to assume values that help the network to correctly predict what it needs to -- in our case, the probable next word in the sentence. This is shown to be very effective in Natural Language Processing tasks, and is a commonplace practice.
<br/><br/>
<font size = 4><strong>
$$Vec("Example") = [0.02, 0.00, 0.00, 0.92, 0.30,...]$$
</font></strong>
<br/>
Word Embedding tends to group up similarly used words *reasonably* together in the vectorial space. For example, if we use T-SNE (a dimensional reduction visualization algorithm) to flatten the dimensions of our vectors into a 2-dimensional space and use the words these vectors represent as their labels, we might see something like this:

<img src=https://ibm.box.com/shared/static/bqhc5dg879gcoabzhxra1w8rkg3od1cu.png width="800"/>
<center>*T-SNE Mockup with clusters marked for easier visualization*</center>

As you can see, words that are frequently used together, in place of each other, or in the same places as them tend to be grouped together -- being closer together the higher these correlations are. For example, "None" is pretty semantically close to "Zero", while a phrase that uses "Italy" can probably also fit "Germany" in it, with little damage to the sentence structure. A vectorial "closeness" for similar words like this is a great indicator of a well-built model.

---
 

We need to import the necessary modules for our code. We need **`numpy` and `tensorflow`**, obviously. Additionally, we can import directly the **`tensorflow.models.rnn.rnn`** model, which includes the function for building RNNs, and **`tensorflow.models.rnn.ptb.reader`** which is the helper module for getting the input data from the dataset we just downloaded.

If you want to learm more take a look at https://www.tensorflow.org/versions/r0.11/api_docs/python/rnn_cell/

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import tensorflow as tf

In [0]:
!mkdir -p /resources/data
!wget -q -O /resources/data/ptb.zip https://ibm.box.com/shared/static/z2yvmhbskc45xd2a9a4kkn6hg4g4kj5r.zip
!unzip -o /resources/data/ptb.zip -d /resources/
!cp /resources/ptb/reader.py .
import reader

Archive:  /resources/data/ptb.zip
  inflating: /resources/ptb/reader.py  
  inflating: /resources/__MACOSX/ptb/._reader.py  
  inflating: /resources/__MACOSX/._ptb  


## Building the LSTM model for Language Modeling
Now that we know exactly what we are doing, we can start building our model using TensorFlow. The very first thing we need to do is download and extract the `simple-examples` dataset, which can be done by executing the code cell below.

In [0]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz 
!tar xzf simple-examples.tgz -C /resources/data/

--2019-06-24 01:54:24--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)... 147.229.9.23, 2001:67c:1220:809::93e5:917
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34869662 (33M) [application/x-gtar]
Saving to: ‘simple-examples.tgz.1’

simple-examples.tgz 100%[===================>]  33.25M  2.91MB/s    in 13s     

2019-06-24 01:54:37 (2.60 MB/s) - ‘simple-examples.tgz.1’ saved [34869662/34869662]



Additionally, for the sake of making it easy to play around with the model's hyperparameters, we can declare them beforehand. Feel free to change these -- you will see a difference in performance each time you change those!  



In [0]:
#Initial weight scale
init_scale = 0.1
#Initial learning rate
learning_rate = 1.0
#Maximum permissible norm for the gradient (For gradient clipping -- another measure against Exploding Gradients)
max_grad_norm = 5
#The number of layers in our model
num_layers = 2
#The total number of recurrence steps, also known as the number of layers when our RNN is "unfolded"
num_steps = 20
#The number of processing units (neurons) in the hidden layers
hidden_size = 200
#The maximum number of epochs trained with the initial learning rate
max_epoch = 4
#The total number of epochs in training
max_max_epoch = 13
#The probability for keeping data in the Dropout Layer (This is an optimization, but is outside our scope for this notebook!)
#At 1, we ignore the Dropout Layer wrapping.
keep_prob = 1
#The decay for the learning rate
decay = 0.5
#The size for each batch of data
batch_size = 30
#The size of our vocabulary
vocab_size = 10000
#Training flag to separate training from testing
is_training = 1
#Data directory for our dataset
data_dir = "/resources/data/simple-examples/data/"

Some clarifications for LSTM architecture based on the argumants:

Network structure:
- In this network, the number of LSTM cells are 2. To give the model more expressive power, we can add multiple layers of LSTMs to process the data. The output of the first layer will become the input of the second and so on.
- The recurrence steps is 20, that is, when our RNN is "Unfolded", the recurrence step is 20.   
- the structure is like: 
     - 200 input units -> [200x200] Weight -> 200 Hidden units (first layer) -> [200x200] Weight matrix  -> 200 Hidden units (second layer) ->  [200] weight Matrix -> 200 unit output

Hidden layer:
- Each LSTM has 200 hidden units which is equivalant to the dimensianality of the embedding words and output. 

Input layer: 
- The network has 200 input units. 
- Suppose each word is represented by an embedding vector of dimensionality e=200. The input layer of each cell will have 200 linear units. These e=200 linear units are connected to each of the h=200 LSTM units in the hidden layer (assuming there is only one hidden layer, though our case has 2 layers).
- The input shape is [batch_size, num_steps], that is [30x20]. It will turn into [30x20x200] after embedding, and then 20x[30x200]




There is a lot to be done and a ton of information to process at the same time, so go over this code slowly. It may seem complex at first, but if you try to ally what you just learned about language modelling to the code you see, you should be able to understand it.

This code is adapted from the [PTBModel](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/models/rnn/ptb/ptb_word_lm.py) example bundled with the TensorFlow source code.


#### Train data
The story starts from data: 
- Train data is a list of words, represented by numbers - N=929589 numbers, e.g. [9971, 9972, 9974, 9975,...]
- We read data as mini-batch of size b=30. Assume the size of each sentence is 20 words (num_steps = 20). Then it will take int(N/b*h)+1=1548 iterations for the learner to go through all sentences once. So, the number of iterators is 1548
- Each batch data is read from train dataset of size 600, and shape of [30x20]

First we start an interactive session:

In [0]:
session=tf.InteractiveSession()

/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, a, b = raw_data

Lets just read one mini-batch now and feed our network:

In [0]:
itera = reader.ptb_iterator(train_data, batch_size, num_steps)
first_touple=itera.next()
x=first_touple[0]
y=first_touple[1]

In [0]:
x.shape

(30, 20)

Lets look at 3 sentences of our input x:

In [0]:
x[0:3]

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605],
       [   0, 1071,    4,    0,  185,   24,  368,   20,   31, 3109,  954,
          12,    3,   21,    2, 2915,    2,   12,    3,   21]],
      dtype=int32)

In [0]:
size = hidden_size

we define 2 place holders to feed them with mini-batchs, that is x and y:

In [0]:
_input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]
_targets = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]

lets defin a dictionary, and use it later to feed the placeholders with our first mini-batch:

In [0]:
feed_dict={_input_data:x, _targets:y}

For example, we can use it to feed _input_data:

In [0]:
session.run(_input_data,feed_dict)

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605],
       [   0, 1071,    4,    0,  185,   24,  368,   20,   31, 3109,  954,
          12,    3,   21,    2, 2915,    2,   12,    3,   21],
       [   3,   71,    4,   27,  246,   60,   11,  215,    4,    1, 1846,
           9,    3,   71,  546,    2, 6505,  162,    6,  104],
       [  93,   25,    6,  261,  681,  251,    0,  278, 3246,   13,  200,
           1,    8,  105, 3360,    1,    4,    0,  536,    4],
       [  20,    6,  954,   12,    3,   21,   78,   14,  977,  726,    0,
          37,   42,   34,    5,  437,  116,  206,  927,    2],
       [  18,  296,    7,  201,   76,    4,  182,  560, 3836,   17,  974,
         975,    6,  942,    4,  156,   77, 1570,  288,  644],
       [  23, 1238,  899,    5,   25,  20

In this step, we create the stacked LSTM, which is a 2 layer LSTM network:

In [0]:
lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, forget_bias=0.0)
stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell] * num_layers)

W0624 01:54:44.076698 140359657342848 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0624 01:54:44.078042 140359657342848 deprecation.py:323] From <ipython-input-27-1ce7ab7d526a>:1: __init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0624 01:54:44.079823 140359657342848 deprecation.py:323] From <ipython-input-27-1ce7ab7d526a>:2: __init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class i

Also, we initialize the states of the nework:

#### _initial_state

For each LCTM, there are 2 state matrics, c_state and m_state.  c_state and m_state represent "Memory State" and "Cell State". Each hidden layer, has a vector of size 30, which keeps the states. so, for 200 hidden units in each LSTM, we have a matrix of size [30x200]

In [0]:
_initial_state = stacked_lstm.zero_state(batch_size, tf.float32)
_initial_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(30, 200) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(30, 200) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros:0' shape=(30, 200) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros_1:0' shape=(30, 200) dtype=float32>))

lets look at the states, though they are all zero for now:

In [0]:
session.run(_initial_state,feed_dict)

(LSTMStateTuple(c=array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), h=array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)),
 LSTMStateTuple(c=array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), h=array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ...,

### Embeddings
We create the embeddings for our input data. embedding is dictionary of [10000x200] for all 10000 unique words.

In [0]:
embedding = tf.get_variable("embedding", [vocab_size, hidden_size])  #[10000x200]

W0624 01:54:44.920933 140359657342848 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
session.run(tf.global_variables_initializer())
session.run(embedding, feed_dict)

array([[ 0.02140534,  0.00550522, -0.01482503, ...,  0.01176408,
        -0.01863301,  0.0055758 ],
       [-0.01267557, -0.01256886,  0.00665567, ...,  0.00721074,
        -0.02053483, -0.00364296],
       [ 0.0212091 , -0.005954  ,  0.01396581, ..., -0.01483609,
        -0.01148561,  0.01904359],
       ...,
       [-0.02359359, -0.01117608, -0.01584147, ..., -0.00691666,
        -0.00358058, -0.0118849 ],
       [ 0.01396932, -0.0055148 ,  0.00656251, ..., -0.00057484,
        -0.02294816,  0.00999484],
       [ 0.01305863, -0.01239078,  0.0101353 , ...,  0.00537324,
        -0.01841567, -0.00238191]], dtype=float32)

embedding_lookup goes to each row of input_data, and for each word in the row/sentence, finds the correspond vector in embedding.
It creates a [30*20*200] matrix, so, the first elemnt of __inputs__ (the first sentence), is a matrix of 20x200, which each row of it is vector representing a word in the sentence.


In [0]:
# Define where to get the data for our embeddings from
inputs = tf.nn.embedding_lookup(embedding, _input_data)  #shape=(30, 20, 200) 

In [0]:
inputs

<tf.Tensor 'embedding_lookup/Identity:0' shape=(30, 20, 200) dtype=float32>

In [0]:
session.run(inputs[0], feed_dict)

array([[-1.0708347e-05, -2.0837609e-02,  1.6509626e-02, ...,
         1.6388316e-02,  9.1803893e-03, -4.2467136e-03],
       [-3.8686767e-03, -1.5326759e-02,  1.2390625e-02, ...,
        -1.7491987e-02,  2.1381341e-02,  7.1007609e-03],
       [-1.5611570e-02, -2.3621557e-02, -2.2025842e-02, ...,
         1.6699381e-02,  1.7022006e-03, -4.2792987e-03],
       ...,
       [-2.0783259e-02, -7.1724225e-03,  4.2724628e-03, ...,
         4.1493662e-03, -9.5144967e-03, -9.2595806e-03],
       [ 2.3578763e-02,  9.7700022e-04, -3.0384902e-03, ...,
        -1.3947031e-02, -6.3981228e-03,  2.4096031e-02],
       [-7.5223725e-03,  2.1261767e-02,  1.5981194e-02, ...,
         1.6029302e-02,  3.6460347e-03, -1.0842546e-02]], dtype=float32)

### Constructing Recurrent Neural Networks
__tf.nn.dynamicrnn()__ creates a recurrent neural network using __stacked_lstm__ which is an instance of RNNCell. 

The input should be a Tensor of shape: [batch_size, max_time, ...], in our case it would be (30, 20, 200)

This method, returns a pair (outputs, new_state) where:
- outputs is a length T list of outputs (one for each input), or a nested tuple of such elements.
- new_state is the final state


In [0]:
outputs, new_state =  tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=_initial_state)

W0624 01:54:45.086965 140359657342848 deprecation.py:323] From <ipython-input-35-f5b68f99f9ac>:1: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0624 01:54:45.476373 140359657342848 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/rnn_cell_impl.py:738: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


so, lets look at the outputs. The output of the stackedLSTM comes from 200 hidden_layer, and in each time step(=20), one of them get activated. we use the linear activation to map the 200 hidden layer to a [?x10 matrix]

In [0]:
outputs

<tf.Tensor 'rnn/transpose_1:0' shape=(30, 20, 200) dtype=float32>

In [0]:
session.run(tf.global_variables_initializer())
session.run(outputs[0], feed_dict)

array([[ 3.0164805e-04,  1.9358080e-04,  5.9345228e-05, ...,
         2.3719223e-04, -5.0356833e-04,  1.9678998e-07],
       [ 6.5629696e-04,  7.1240275e-04, -2.8955319e-04, ...,
         2.3138465e-04, -7.1208132e-04, -4.9436552e-04],
       [ 6.6113996e-04,  6.7483855e-04, -3.4164049e-04, ...,
        -3.4497898e-05,  9.5482159e-05,  1.5332083e-04],
       ...,
       [ 3.0820695e-04,  5.7327200e-04, -1.0496346e-04, ...,
         1.9396290e-04,  6.4874743e-04, -9.6199386e-05],
       [ 3.6304994e-04,  5.0525932e-04, -5.0771324e-04, ...,
         1.2474939e-04,  1.5257637e-04, -2.7159177e-04],
       [ 2.2463142e-05,  5.0882337e-04, -1.4450643e-04, ...,
         2.9499517e-05, -4.8898970e-04, -5.1762897e-04]], dtype=float32)

Lets reshape the output tensor from  [30 x 20 x 200] to [600 x 200]

In [0]:
output = tf.reshape(outputs, [-1, size])
output

<tf.Tensor 'Reshape:0' shape=(600, 200) dtype=float32>

In [0]:
session.run(output[0], feed_dict)

array([ 3.01648048e-04,  1.93580796e-04,  5.93452278e-05, -7.77744353e-05,
       -6.07549664e-05, -3.09731869e-04,  4.57707589e-04, -4.26303915e-04,
        1.09800720e-03,  1.61425167e-04,  1.59318777e-04,  1.14588278e-04,
        2.71049037e-04,  4.01778270e-05,  1.21103476e-04, -1.57823277e-04,
        4.49903542e-04, -3.27737682e-04,  4.90641629e-04, -2.19404043e-04,
       -9.16074787e-05, -5.16200787e-04, -4.17364674e-04,  5.26059943e-04,
       -6.44039319e-05,  4.21113917e-04,  7.86803357e-05, -2.45695555e-04,
       -2.21353155e-04,  2.38128472e-04, -1.25268067e-04,  2.75905739e-04,
        1.25949708e-04, -1.32864516e-04, -2.69225711e-04, -1.72758723e-06,
       -4.56721958e-04,  1.01490550e-04, -3.75757430e-04, -4.66081292e-05,
        3.13406868e-04,  1.25724458e-04, -2.64512462e-04, -1.12718881e-04,
       -3.27234098e-04,  4.58967581e-04,  1.78280079e-05, -3.59436781e-05,
        2.50195502e-04, -1.20749664e-05, -2.80397100e-04, -4.25128703e-04,
       -2.05705015e-04,  

### logistic unit
Now, we create a logistic unit to return the probability of the output word. That is, mapping the 600

Softmax = [600 x 200]* [200 x 1000]+ [1 x 1000] -> [600 x 1000]

In [0]:
softmax_w = tf.get_variable("softmax_w", [size, vocab_size]) #[200x1000]
softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x1000]
logits = tf.matmul(output, softmax_w) + softmax_b

In [0]:
session.run(tf.global_variables_initializer())
logi = session.run(logits, feed_dict)
logi.shape

(600, 10000)

In [0]:
First_word_output_probablity = logi[0]
First_word_output_probablity.shape

(10000,)

### Prediction
The maximum probablity

In [0]:
embedding_array= session.run(embedding, feed_dict)
np.argmax(First_word_output_probablity)

3996

So, what is the ground truth for the first word of first sentence? 

In [0]:
y[0][0]

9971

Also, you can get it from target tensor, if you want to find the embedding vector:

In [0]:
_targets

<tf.Tensor 'Placeholder_1:0' shape=(30, 20) dtype=int32>

It is time to compare logit with target

In [0]:
targ = session.run(tf.reshape(_targets, [-1]), feed_dict) 

In [0]:
first_word_target_code= targ[0]
first_word_target_code

9971

In [0]:
first_word_target_vec = session.run( tf.nn.embedding_lookup(embedding, targ[0]))
first_word_target_vec

array([ 1.45452209e-02, -1.30262608e-02, -1.98408663e-02, -1.35601358e-02,
       -1.99249685e-02,  3.35625000e-03,  4.88178246e-03,  6.78397156e-03,
       -2.26401929e-02,  1.32226758e-03, -1.29080601e-02, -2.20010933e-02,
       -4.61256690e-03,  1.31180398e-02,  1.82501823e-02, -2.39613466e-03,
       -2.09959503e-02,  1.61860883e-03, -1.42631894e-02,  1.71613544e-02,
       -4.61905450e-03,  2.31974870e-02, -4.79087606e-03,  8.59331340e-03,
        7.01745227e-03,  3.87981534e-03,  1.06820241e-02, -1.33737829e-02,
       -1.94310788e-02, -3.88325006e-03, -5.66144288e-03, -1.26283951e-02,
        5.39467297e-03,  2.13354528e-02, -6.91988319e-03,  1.05152428e-02,
       -2.37712841e-02, -1.45483417e-02, -8.37401301e-03, -1.42862964e-02,
       -1.89849939e-02, -2.20096745e-02,  1.28610097e-02, -5.32534160e-03,
        1.56600438e-02, -1.75520889e-02,  5.53707406e-03,  1.19946897e-02,
       -8.01094249e-03, -7.65705295e-03, -1.56114772e-02, -4.11813520e-03,
       -2.94980407e-03, -

#### Objective function

Now we want to define our objective function. Our objective is to minimize loss function, that is, to minimize the average negative log probability of the target words:

loss=−1N∑i=1Nln⁡ptargeti  
This function is already implimented and available in TensorFlow through __sequence_loss_by_example__ so we can just use it here. __sequence_loss_by_example__ is weighted cross-entropy loss for a sequence of logits (per example).  

Its arguments:  

logits: List of 2D Tensors of shape [batch_size x num_decoder_symbols].  
targets: List of 1D batch-sized int32 Tensors of the same length as logits.  
weights: List of 1D batch-sized float-Tensors of the same length as logits.  

In [0]:
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(_targets, [-1])],[tf.ones([batch_size * num_steps])])

loss is a 1D batch-sized float Tensor [600x1]: The log-perplexity for each sequence.

In [0]:
session.run(loss, feed_dict)

array([9.208332 , 9.223998 , 9.202899 , 9.207659 , 9.207867 , 9.22586  ,
       9.204422 , 9.213485 , 9.223234 , 9.212697 , 9.19851  , 9.213982 ,
       9.213981 , 9.204129 , 9.212322 , 9.22629  , 9.209499 , 9.221797 ,
       9.212761 , 9.196641 , 9.213057 , 9.194944 , 9.227493 , 9.200231 ,
       9.218625 , 9.205187 , 9.219167 , 9.20751  , 9.214114 , 9.197215 ,
       9.214044 , 9.202149 , 9.2013035, 9.224961 , 9.212859 , 9.193695 ,
       9.201193 , 9.201054 , 9.220659 , 9.209602 , 9.210301 , 9.200089 ,
       9.194098 , 9.196737 , 9.194475 , 9.222182 , 9.200486 , 9.209416 ,
       9.203566 , 9.201269 , 9.22086  , 9.218668 , 9.199681 , 9.225093 ,
       9.225205 , 9.22506  , 9.220655 , 9.218466 , 9.199634 , 9.199977 ,
       9.200053 , 9.200109 , 9.225645 , 9.215298 , 9.220671 , 9.213959 ,
       9.194421 , 9.199904 , 9.218541 , 9.209958 , 9.212626 , 9.218594 ,
       9.200028 , 9.20829  , 9.2249155, 9.20206  , 9.226015 , 9.213027 ,
       9.195386 , 9.193154 , 9.194746 , 9.213053 , 

In [0]:
cost = tf.reduce_sum(loss) / batch_size

session.run(tf.global_variables_initializer())
session.run(cost, feed_dict)


184.20721

Now, lets store the new state as final state 

In [0]:
# 
final_state = new_state

### Training

To do gradient clipping in TensorFlow we have to take the following steps:

1. Define the optimizer.
2. Extract variables that are trainable.
3. Calculate the gradients based on the loss function.
4. Apply the optimizer to the variables / gradients tuple.

#### 1. Define Optimizer

__GradientDescentOptimizer__ constructs a new gradient descent optimizer. Later, we use constructed __optimizer__ to compute gradients for a loss and apply gradients to variables.

In [0]:
# Create a variable for the learning rate
lr = tf.Variable(0.0, trainable=False)
# Create the gradient descent optimizer with our learning rate
optimizer = tf.train.GradientDescentOptimizer(lr)


#### 2. Trainable Variables

Definining a variable, if you passed _trainable=True_, the _Variable()_ constructor automatically adds new variables to the graph collection __GraphKeys.TRAINABLE_VARIABLES__. Now, using _tf.trainable_variables()_ you can get all variables created with __trainable=True__.

In [0]:
# Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'embedding:0' shape=(10000, 200) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(400, 800) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'softmax_w:0' shape=(200, 10000) dtype=float32_ref>,
 <tf.Variable 'softmax_b:0' shape=(10000,) dtype=float32_ref>]

we can find the name and scope of all variables:

In [0]:
tvars=tvars[3:]

In [0]:
[v.name for v in tvars]

[u'softmax_w:0', u'softmax_b:0']

#### 3. Calculate the gradients based on the loss function

In [0]:
cost

<tf.Tensor 'truediv:0' shape=() dtype=float32>

In [0]:
tvars

[<tf.Variable 'softmax_w:0' shape=(200, 10000) dtype=float32_ref>,
 <tf.Variable 'softmax_b:0' shape=(10000,) dtype=float32_ref>]

#### Gradient:
The gradient of a function (line) is the slope of the line, or the rate of change of a function. It's a vector (a direction to move) that points in the direction of greatest increase of the function, and calculated by __derivative__ operation.

First lets recall the gradient function using an toy example:
$$ z=\left(2x^2+3xy\right)$$

In [0]:
var_x = tf.placeholder(tf.float32)
var_y = tf.placeholder(tf.float32) 
func_test = 2.0*var_x*var_x + 3.0*var_x*var_y
session.run(tf.global_variables_initializer())
feed={var_x:1.0,var_y:2.0}
session.run(func_test, feed)


8.0

The __tf.gradients()__ function allows you to compute the symbolic gradient of one tensor with respect to one or more other tensors—including variables. __tf.gradients(func,xs)__ constructs symbolic partial derivatives of sum of __func__ w.r.t. _x_ in __xs__. 

Now, lets look at the derivitive w.r.t. __var_x__:
$$ \frac{\partial \:}{\partial \:x}\left(2x^2+3xy\right)=4x+3y $$


In [0]:
var_grad = tf.gradients(func_test, [var_x])
session.run(var_grad,feed)

[10.0]

the derivitive w.r.t. __var_y__:
$$ \frac{\partial \:}{\partial \:x}\left(2x^2+3xy\right)=3x $$

In [0]:
var_grad = tf.gradients(func_test, [var_y])
session.run(var_grad,feed)

[3.0]

Now, we can look at gradients w.r.t all variables:

In [0]:
tf.gradients(cost, tvars)

[<tf.Tensor 'gradients_2/MatMul_grad/MatMul_1:0' shape=(200, 10000) dtype=float32>,
 <tf.Tensor 'gradients_2/add_grad/Reshape_1:0' shape=(10000,) dtype=float32>]

In [0]:
grad_t_list = tf.gradients(cost, tvars)
#sess.run(grad_t_list,feed_dict)




now, we have a list of tensors, t-list. We can use it to find clipped tensors. __clip_by_global_norm__ clips values of multiple tensors by the ratio of the sum of their norms.

__clip_by_global_norm__ get _t-list_ as input and returns 2 things:
 - a list of clipped tensors, so called _list_clipped_ 
 - the global norm (global_norm) of all tensors in t_list

In [0]:
max_grad_norm

5

In [0]:
# Define the gradient clipping threshold
grads, _ = tf.clip_by_global_norm(grad_t_list, max_grad_norm)
grads

W0624 01:54:47.485759 140359657342848 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/clip_ops.py:286: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[<tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_0:0' shape=(200, 10000) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_1:0' shape=(10000,) dtype=float32>]

In [0]:
session.run(grads,feed_dict)

[array([[-2.23864525e-04, -2.84448877e-04, -3.70307331e-04, ...,
          3.98499736e-07,  3.97502873e-07,  4.01247746e-07],
        [ 9.49665700e-05,  4.36206683e-05,  9.82029960e-05, ...,
         -2.66120594e-07, -2.65446374e-07, -2.67918892e-07],
        [ 9.85850638e-05,  1.97140209e-04,  3.24608845e-04, ...,
         -3.21983947e-07, -3.21160144e-07, -3.24145191e-07],
        ...,
        [-1.24243801e-04,  3.11763870e-06,  5.68766700e-05, ...,
          1.16034542e-07,  1.15766078e-07,  1.16871234e-07],
        [-7.84041331e-05, -1.52070861e-05,  4.41127522e-05, ...,
         -1.18427465e-07, -1.18143539e-07, -1.19224239e-07],
        [-4.93611005e-05,  1.61433527e-05, -5.51462581e-05, ...,
         -3.29592211e-08, -3.28409726e-08, -3.31344978e-08]], dtype=float32),
 array([-0.7979654 , -1.0313638 , -1.0313312 , ...,  0.0019881 ,
         0.00198312,  0.00200159], dtype=float32)]

#### 4. Apply the optimizer to the variables / gradients tuple.

In [0]:
# Create the training TensorFlow Operation through our optimizer
train_op = optimizer.apply_gradients(zip(grads, tvars))

In [0]:
session.run(tf.global_variables_initializer())
session.run(train_op,feed_dict)

We learned how the model is build step by step. Noe, let's then create a Class that represents our model. This class needs a few things:
- We have to create the model in accordance with our defined hyperparameters
- We have to create the placeholders for our input data and expected outputs (the real data)
- We have to create the LSTM cell structure and connect them with our RNN structure
- We have to create the word embeddings and point them to the input data
- We have to create the input structure for our RNN
- We have to instanciate our RNN model and retrieve the variable in which we should expect our outputs to appear
- We need to create a logistic structure to return the probability of our words
- We need to create the loss and cost functions for our optimizer to work, and then create the optimizer
- And finally, we need to create a training operation that can be run to actually train our model


In [0]:
class PTBModel(object):

    def __init__(self, is_training):
        ######################################
        # Setting parameters for ease of use #
        ######################################
        self.batch_size = batch_size
        self.num_steps = num_steps
        size = hidden_size
        self.vocab_size = vocab_size
        
        ###############################################################################
        # Creating placeholders for our input data and expected outputs (target data) #
        ###############################################################################
        self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]
        self._targets = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]

        ##########################################################################
        # Creating the LSTM cell structure and connect it with the RNN structure #
        ##########################################################################
        # Create the LSTM unit. 
        # This creates only the structure for the LSTM and has to be associated with a RNN unit still.
        # The argument n_hidden(size=200) of BasicLSTMCell is size of hidden layer, that is, the number of hidden units of the LSTM (inside A).
        # Size is the same as the size of our hidden layer, and no bias is added to the Forget Gate. 
        # LSTM cell processes one word at a time and computes probabilities of the possible continuations of the sentence.
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(size, forget_bias=0.0)
        
        # Unless you changed keep_prob, this won't actually execute -- this is a dropout wrapper for our LSTM unit
        # This is an optimization of the LSTM output, but is not needed at all
        if is_training and keep_prob < 1:
            lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
        
        # By taking in the LSTM cells as parameters, the MultiRNNCell function junctions the LSTM units to the RNN units.
        # RNN cell composed sequentially of multiple simple cells.
        stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell] * num_layers)

        # Define the initial state, i.e., the model state for the very first data point
        # It initialize the state of the LSTM memory. The memory state of the network is initialized with a vector of zeros and gets updated after reading each word.
        self._initial_state = stacked_lstm.zero_state(batch_size, tf.float32)

        ####################################################################
        # Creating the word embeddings and pointing them to the input data #
        ####################################################################
        with tf.device("/cpu:0"):
            # Create the embeddings for our input data. Size is hidden size.
            embedding = tf.get_variable("embedding", [vocab_size, size])  #[10000x200]
            # Define where to get the data for our embeddings from
            inputs = tf.nn.embedding_lookup(embedding, self._input_data)

        # Unless you changed keep_prob, this won't actually execute -- this is a dropout addition for our inputs
        # This is an optimization of the input processing and is not needed at all
        if is_training and keep_prob < 1:
            inputs = tf.nn.dropout(inputs, keep_prob)

        ############################################
        # Creating the input structure for our RNN #
        ############################################
        # Input structure is 20x[30x200]
        # Considering each word is represended by a 200 dimentional vector, and we have 30 batchs, we create 30 word-vectors of size [30xx2000]
        #inputs = [tf.squeeze(input_, [1]) for input_ in tf.split(1, num_steps, inputs)]
        # The input structure is fed from the embeddings, which are filled in by the input data
        # Feeding a batch of b sentences to a RNN:
        # In step 1,  first word of each of the b sentences (in a batch) is input in parallel.  
        # In step 2,  second word of each of the b sentences is input in parallel. 
        # The parallelism is only for efficiency.  
        # Each sentence in a batch is handled in parallel, but the network sees one word of a sentence at a time and does the computations accordingly. 
        # All the computations involving the words of all sentences in a batch at a given time step are done in parallel. 

        ####################################################################################################
        # Instanciating our RNN model and retrieving the structure for returning the outputs and the state #
        ####################################################################################################
        
        outputs, state = tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=self._initial_state)

        #########################################################################
        # Creating a logistic unit to return the probability of the output word #
        #########################################################################
        output = tf.reshape(outputs, [-1, size])
        softmax_w = tf.get_variable("softmax_w", [size, vocab_size]) #[200x1000]
        softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x1000]
        logits = tf.matmul(output, softmax_w) + softmax_b

        #########################################################################
        # Defining the loss and cost functions for the model's learning to work #
        #########################################################################
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(self._targets, [-1])],
                                                      [tf.ones([batch_size * num_steps])])
        self._cost = cost = tf.reduce_sum(loss) / batch_size

        # Store the final state
        self._final_state = state

        #Everything after this point is relevant only for training
        if not is_training:
            return

        #################################################
        # Creating the Training Operation for our Model #
        #################################################
        # Create a variable for the learning rate
        self._lr = tf.Variable(0.0, trainable=False)
        # Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
        tvars = tf.trainable_variables()
        # Define the gradient clipping threshold
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), max_grad_norm)
        # Create the gradient descent optimizer with our learning rate
        optimizer = tf.train.GradientDescentOptimizer(self.lr)
        # Create the training TensorFlow Operation through our optimizer
        self._train_op = optimizer.apply_gradients(zip(grads, tvars))

    # Helper functions for our LSTM RNN class

    # Assign the learning rate for this model
    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))

    # Returns the input data for this model at a point in time
    @property
    def input_data(self):
        return self._input_data

    # Returns the targets for this model at a point in time
    @property
    def targets(self):
        return self._targets

    # Returns the initial state for this model
    @property
    def initial_state(self):
        return self._initial_state

    # Returns the defined Cost
    @property
    def cost(self):
        return self._cost

    # Returns the final state for this model
    @property
    def final_state(self):
        return self._final_state

    # Returns the current learning rate for this model
    @property
    def lr(self):
        return self._lr

    # Returns the training operation defined for this model
    @property
    def train_op(self):
        return self._train_op

With that, the actual structure of our Recurrent Neural Network with Long Short-Term Memory is finished. What remains for us to do is to actually create the methods to run through time -- that is, the `run_epoch` method to be run at each epoch and a `main` script which ties all of this together.

What our `run_epoch` method should do is take our input data and feed it to the relevant operations. This will return at the very least the current result for the cost function.

In [0]:
##########################################################################################################################
# run_epoch takes as parameters the current session, the model instance, the data to be fed, and the operation to be run #
##########################################################################################################################
def run_epoch(session, m, data, eval_op, verbose=False):

    #Define the epoch size based on the length of the data, batch size and the number of steps
    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0
    #state = m.initial_state.eval()
    #m.initial_state = tf.convert_to_tensor(m.initial_state) 
    #state = m.initial_state.eval()
    state = session.run(m.initial_state)
    
    #For each step and data point
    for step, (x, y) in enumerate(reader.ptb_iterator(data, m.batch_size, m.num_steps)):
 
        #Evaluate and return cost, state by running cost, final_state and the function passed as parameter
        cost, state, _ = session.run([m.cost, m.final_state, eval_op],
                                     {m.input_data: x,
                                      m.targets: y,
                                      m.initial_state: state})
        
        #Add returned cost to costs (which keeps track of the total costs for this epoch)
        costs += cost
        
        #Add number of steps to iteration counter
        iters += m.num_steps

        if verbose and step % (epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" % (step * 1.0 / epoch_size, np.exp(costs / iters),
              iters * m.batch_size / (time.time() - start_time)))

    # Returns the Perplexity rating for us to keep track of how the model is evolving
    return np.exp(costs / iters)


Now, we create the `main` method to tie everything together. The code here reads the data from the directory, using the `reader` helper module, and then trains and evaluates the model on both a testing and a validating subset of data.

In [0]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, a, b = raw_data

In [0]:
#Initializes the Execution Graph and the Session
with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-init_scale,init_scale)
    
    # Instantiates the model for training
    # tf.variable_scope add a prefix to the variables created with tf.get_variable
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel(is_training=True)
        
    # Reuses the trained parameters for the validation and testing models
    # They are different instances but use the same variables for weights and biases, they just don't change when data is input
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel(is_training=False)
        mtest = PTBModel(is_training=False)

    #Initialize all variables
    tf.global_variables_initializer().run()

    for i in range(max_max_epoch):
        # Define the decay for this epoch
        lr_decay = decay ** max(i - max_epoch, 0.0)
        
        # Set the decayed learning rate as the learning rate for this epoch
        m.assign_lr(session, learning_rate * lr_decay)

        print("Epoch %d : Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        
        # Run the loop for this epoch in the training model
        train_perplexity = run_epoch(session, m, train_data, m.train_op,
                                   verbose=True)
        print("Epoch %d : Train Perplexity: %.3f" % (i + 1, train_perplexity))
        
        # Run the loop for this epoch in the validation model
        valid_perplexity = run_epoch(session, mvalid, valid_data, tf.no_op())
        print("Epoch %d : Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
    
    # Run the loop in the testing model to see how effective was our training
    test_perplexity = run_epoch(session, mtest, test_data, tf.no_op())
    
    print("Test Perplexity: %.3f" % test_perplexity)

Epoch 1 : Learning rate: 1.000
0.006 perplexity: 5299.572 speed: 2313 wps


KeyboardInterrupt: ignored

As you can see, the model's perplexity rating drops very quickly after a few iterations. As was elaborated before, **lower Perplexity means that the model is more certain about its prediction**. As such, we can be sure that this model is performing well!

-------

This is the end of the **Applying Recurrent Neural Networks to Text Processing** notebook. Hopefully you now have a better understanding of Recurrent Neural Networks and how to implement one utilizing TensorFlow. Thank you for reading this notebook, and good luck on your studies.

# Thanks for completing this lesson!

Created by <a href="https://br.linkedin.com/in/walter-gomes-de-amorim-junior-624726121">Walter Gomes de Amorim Junior</a>, <a href = "https://linkedin.com/in/saeedaghabozorgi"> Saeed Aghabozorgi </a></h4>

<hr>

<p>Copyright &copy; 2017 IBM <a href="https://cognitiveclass.ai/?utm_source=ML0151&utm_medium=lab&utm_campaign=cclab">IBM Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://cognitiveclass.ai/mit-license/">MIT License</a>.</p>